# 11.策略梯度算法原理
> Q-learning、DQN 及 DQN 改进算法都是 **基于价值（value-based）** 的方法，在强化学习中，除了基于值函数的方法，还有一支非常经典的方法：**基于策略（policy-based）** 的方法。

- **基于值函数的方法** 主要是学习值函数，然后根据值函数导出一个策略，学习过程中并不存在一个显式的策略
- **基于策略的方法** 则是直接显式地学习一个目标策略

其中 **策略梯度** 是基于策略的方法的基础
它的核心思想是：通过对 **策略的参数化** 表示，利用梯度上升法直接优化 **期望回报**

## 策略梯度
### 策略与期望回报：
> 在强化学习中，策略定义了智能体在某个状态下选择动作的概率分布。假设一个策略是通过参数化的形式表示的，通常写作：

$$\pi_\theta(a|s)$$

- 表示在状态 s 下选择动作 a 的概率，而 θ 是策略的参数（如神经网络的权重）
- 通过调整参数 θ，使得智能体的长期回报（即累积奖励）最大化，即 **最大化期望回报**

> 在某个策略$\pi_\theta(a|s)$下，**期望回报** 可以表示为：

$$J(\theta)=\mathbb{E}_{\pi_\theta}\left[\sum_{t=0}^T\gamma^tr_t\right]$$


- $r_t$是时间步$t$的即时奖励，$\gamma$是折扣因子，$T$是时间步数
- 这其中的整个序列动作的选择都是基于策略$\pi_\theta(a|s)$

> 若熟知 [3.马尔可夫决策过程](../RL_Fundamentals/3.马尔可夫决策过程(Markov_decision_process,MDP).ipynb) 中的一系列定义与公式推导，可知在某个 **确定策略** 下 **期望回报** 亦可视为:

$$J(\theta)=\mathbb{E}_{s_0}[V^{\pi_\theta}(s_0)]$$

- $s_0$表示某个初始状态

> 且：
$$V^\pi(s)=\sum_{a\in A}\pi(a|s)Q^\pi(s,a)$$
$$Q^\pi(s,a)=r(s,a)+\gamma\sum_{s^{\prime}\in S}P(s^{\prime}|s,a)V^\pi(s^{\prime})$$

### 先对状态价值函数求梯度:
>$$\begin{aligned}
\nabla_{\theta}V^{\pi_{\theta}}(s) & =\nabla_\theta(\sum_{a\in A}\pi_\theta(a|s)Q^{\pi_\theta}(s,a)) \\
 & =\sum_{a\in A}(\nabla_\theta\pi_\theta(a|s)Q^{\pi_\theta}(s,a)+\pi_\theta(a|s)\nabla_\theta Q^{\pi_\theta}(s,a)) \\
 & =\sum_{a\in A}(\nabla_\theta\pi_\theta(a|s)Q^{\pi_\theta}(s,a)+\gamma\pi_\theta(a|s)\sum_{s^{\prime}\in S}P(s^{\prime}|s,a)\nabla_\theta V^{\pi_\theta}(s^{\prime})) 
\end{aligned}$$


令 $\phi(s)=\sum_{a\in A}\nabla_{\theta}\pi_{\theta}(a|s)Q^{\pi_{\theta}}(s,a)$
定义 $d^{\pi_\theta}(s\to x,k)$ 为策略$\pi_\theta$从状态s出发k步后到达状态x的概率

>$$\begin{aligned}
 & =\phi(s)+\gamma\sum_a\pi_\theta(a|s)\sum_{s^{\prime}\in S}P(s^{\prime}|s,a)\nabla_\theta V^{\pi_\theta}(s^{\prime}) \\
 & =\phi(s)+\gamma\sum_a\sum_{s^{\prime}\in S}\pi_\theta(a|s)P(s^{\prime}|s,a)\nabla_\theta V^{\pi_\theta}(s^{\prime}) \\
 & =\phi(s)+\gamma\sum_{s^{\prime}\in S}d^{\pi_\theta}(s\to s^{\prime},1)\nabla_\theta V^{\pi_\theta}(s^{\prime}) \\
 & =\phi(s)+\gamma\sum_{s^{\prime}\in S}d^{\pi_{\theta}}(s\to s^{\prime},1)[\phi(s^{\prime})+\gamma\sum_{s^{\prime\prime}\in S}d^{\pi_{\theta}}(s^{\prime}\to s^{\prime\prime},1)\nabla_{\theta}V^{\pi_{\theta}}(s^{\prime\prime})] \\
 & =\phi(s)+\gamma\sum_{s^{\prime}\in S}d^{\pi_{\theta}}(s\to s^{\prime},1)\phi(s^{\prime})+\gamma^{2}\sum_{s^{\prime\prime}\in S}d^{\pi_{\theta}}(s\to s^{\prime\prime},2)\nabla_{\theta}V^{\pi_{\theta}}(s^{\prime\prime}) \\
 & =\phi(s)+\gamma\sum_{s^{\prime}\in S}d^{\pi_{\theta}}(s\to s^{\prime},1)\phi(s^{\prime})+\gamma^{2}\sum_{s^{\prime\prime}\in S}d^{\pi_{\theta}}(s^{\prime}\to s^{\prime\prime},2)\phi(s^{\prime\prime})+\gamma^{3}\sum_{s^{\prime\prime}\in S}d^{\pi_{\theta}}(s\to s^{\prime\prime\prime},3)\nabla_{\theta}V^{\pi_{\theta}}(s^{\prime\prime\prime}) \\
 & =\ldots \\
 & =\sum_{x\in S}\sum_{k=0}^\infty\gamma^kd^{\pi_\theta}(s\to x,k)\phi(x)
\end{aligned}$$

### 再对期望回报求梯度:
>$$\begin{aligned}
\nabla_{\theta}J(\theta) & =\nabla_\theta\mathbb{E}_{s_0}[V^{\pi_\theta}(s_0)] \\
 & =\sum_{s\in S}\mathbb{E}_{s_0}\left[\sum_{k=0}^\infty\gamma^kd^{\pi_\theta}(s_0\to s,k)\right]\phi(s)
\end{aligned}$$

令 $\eta(s)=\mathbb{E}_{s_0}\left[\sum_{k=0}^\infty\gamma^kd^{\pi_0}(s_0\to s,k)\right]$，表示在策略$\pi_{\theta}$下，从初始状态$s_{0}$开始，经过一系列动作后，状态$s$被访问的概率
所以：
- $\sum_s\eta(s)$ 是所有状态的加权概率的总和，可视为当前策略空间的状态总分布（为非负数）
- $\frac{\eta(s)}{\sum_s\eta(s)}$ 表示在策略$\pi_{\theta}$下，状态$s$的访问频率，即[状态访问分布（state visitation distribution）$\nu^{\pi_{\theta}}(s)$](../RL_Fundamentals/1.初探.ipynb)
- $\phi(s)=\sum_{a\in A}\nabla_{\theta}\pi_{\theta}(a|s)Q^{\pi_{\theta}}(s,a)$ 表示经过状态$s$时的价值趋向

> $$=\sum_s\eta(s)\phi(s)$$
> 归一化:
$$=\left(\sum_s\eta(s)\right)\sum_s\frac{\eta(s)}{\sum_s\eta(s)}\sum_{a\in A}\nabla_{\theta}\pi_{\theta}(a|s)Q^{\pi_{\theta}}(s,a)$$
> 正比于:
$$\propto\sum_s\frac{\eta(s)}{\sum_s\eta(s)}\sum_{a\in A}\nabla_{\theta}\pi_{\theta}(a|s)Q^{\pi_{\theta}}(s,a)$$
$$=\sum_s\frac{\eta(s)}{\sum_s\eta(s)}\sum_{a\in A}\pi_\theta(a|s)Q^{\pi_\theta}(s,a)\frac{\nabla_\theta\pi_\theta(a|s)}{\pi_\theta(a|s)}$$
$$=\sum_s\nu^{\pi_{\theta}}(s)\sum_{a\in A}\pi_\theta(a|s)Q^{\pi_\theta}(s,a)\frac{\nabla_\theta\pi_\theta(a|s)}{\pi_\theta(a|s)}$$

**对数技巧（log-derivative trick）**(梯度数值弱化,使更新更稳定)
因为：$\frac{d}{dx}\log f(x)=\frac{1}{f(x)}\cdot\frac{d}{dx}f(x)$
所以：$\nabla_\theta\pi_\theta(a|s)=\pi_\theta(a|s)\nabla_\theta\log\pi_\theta(a|s)$

> $$=\sum_s\nu^{\pi_{\theta}}(s)\sum_{a\in A}\pi_\theta(a|s)Q^{\pi_\theta}(s,a)\nabla_\theta\log\pi_\theta(a|s)$$
> 在策略$\pi_{\theta}$下，状态访问分布$\nu^{\pi_{\theta}}(s)$和$\pi_{\theta}(a|s)$确定，将原始的加权和形式转换为了期望的形式：
> $$=\mathbb{E}_{\pi_\theta}\left[Q^{\pi_\theta}(s,a)\nabla_\theta\log\pi_\theta(a|s)\right]$$

可见，策略梯度算法需要使用当前策略$\pi_{\theta}$ **采样数据** 估计 $Q^{\pi_{\theta}}(s,a)$ 来计算梯度（在线策略（on-policy）算法）